# Results Analysis

1. Best combinaison






In [57]:
import pandas as pd
import pickle as pk
from pipeline_config import list_of_models, list_of_prompts, quantizations
import re

prediction = pd.read_csv(f"outputs/cities_prediction.csv")
common_columns = ["Geoname ID", "Name", "ASCII Name", "Alternate Names", "Feature Class", "Feature Code", "Country Code", "Country name EN", "Country Code 2", "Admin1 Code", "Admin2 Code", "Admin3 Code", "Admin4 Code", "Population", "Elevation", "DIgital Elevation Model", "Timezone", "Modification date", "LABEL EN", "Coordinates", "city"]


/tmp/ipykernel_3663659/3382307164.py:6: DtypeWarning: Columns (10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  prediction = pd.read_csv(f"outputs/cities_prediction.csv")


## 1. Best / Worst combinaisons

Distance min/max/average by combinaison

In [38]:
import numpy as np

# Compute mean, standard deviation, and count of non-NaN values for columns containing "distance"
distance_stats = {
    k: (prediction[k].mean(), prediction[k].std(), prediction[k].count()) 
    for k in prediction.keys() if "distance" in k
}

# Sort columns by their mean value
sorted_columns = sorted(distance_stats.items(), key=lambda item: item[1][0])

# Select the 5 best (lowest mean) and 5 worst (highest mean) columns
best_5 = sorted_columns[:5]   # Top 5 best (lowest mean)
worst_5 = sorted_columns[-5:] # Top 5 worst (highest mean)

# Display results
print("🏆 **Top 5 Best Columns** (Lowest Mean):")
for col, (mean, std, count) in best_5:
    print(f"{col} | Mean: {mean:.4f} | STD: {std:.4f} | Non-NaN values: {count}")

print("\n💀 **Top 5 Worst Columns** (Highest Mean):")
for col, (mean, std, count) in reversed(worst_5):  # Reverse to show the worst first
    print(f"{col} | Mean: {mean:.4f} | STD: {std:.4f} | Non-NaN values: {count}")


🏆 **Top 5 Best Columns** (Lowest Mean):
mistralai/Mistral-Small-24B-Base-2501_float16_gps_fr_distance | Mean: 87.0070 | STD: 672.1141 | Non-NaN values: 904
meta-llama/Llama-3.2-3B_int4_gps_fr_distance | Mean: 88.6713 | STD: 187.2747 | Non-NaN values: 38
meta-llama/Llama-2-13b-hf_int8_gps_fr_distance | Mean: 108.8562 | STD: 168.5298 | Non-NaN values: 35
mistralai/Mistral-Small-24B-Base-2501_int8_gps_fr_distance | Mean: 113.4336 | STD: 830.8944 | Non-NaN values: 792
Qwen/Qwen2.5-14B_float16_gps_fr_distance | Mean: 114.9181 | STD: 653.1495 | Non-NaN values: 984

💀 **Top 5 Worst Columns** (Highest Mean):
meta-llama/Llama-3.1-8B-Instruct_float16_gps_fr_distance | Mean: 6290.6920 | STD: 5321.7150 | Non-NaN values: 607
meta-llama/Llama-2-13b-chat-hf_float16_gps_fr_distance | Mean: 6127.1839 | STD: 4790.6469 | Non-NaN values: 971
meta-llama/Llama-3.1-8B-Instruct_int8_gps_fr_distance | Mean: 6050.3589 | STD: 5063.5476 | Non-NaN values: 757
meta-llama/Llama-2-13b-chat-hf_int8_gps_fr_distance | M

**Remove columns when there is not enough distance computed (~750)**

In [39]:
df = prediction.copy()
df = df.dropna(axis=1, thresh=len(df) - 250)

distance_stats = {
    k: (df[k].mean(), df[k].std(), df[k].count()) 
    for k in df.keys() if "distance" in k
}

# Sort columns by their mean value
sorted_columns = sorted(distance_stats.items(), key=lambda item: item[1][0])

# Select the 5 best (lowest mean) and 5 worst (highest mean) columns
best_5 = sorted_columns[:5]   # Top 5 best (lowest mean)
worst_5 = sorted_columns[-5:] # Top 5 worst (highest mean)

# Display results
print("🏆 **Top 5 Best Columns** (Lowest Mean):")
for col, (mean, std, count) in best_5:
    print(f"{col} | Mean: {mean:.4f} | STD: {std:.4f} | Non-NaN values: {count}")

print("\n💀 **Top 5 Worst Columns** (Highest Mean):")
for col, (mean, std, count) in reversed(worst_5):  # Reverse to show the worst first
    print(f"{col} | Mean: {mean:.4f} | STD: {std:.4f} | Non-NaN values: {count}")

🏆 **Top 5 Best Columns** (Lowest Mean):
mistralai/Mistral-Small-24B-Base-2501_float16_gps_fr_distance | Mean: 87.0070 | STD: 672.1141 | Non-NaN values: 904
mistralai/Mistral-Small-24B-Base-2501_int8_gps_fr_distance | Mean: 113.4336 | STD: 830.8944 | Non-NaN values: 792
Qwen/Qwen2.5-14B_float16_gps_fr_distance | Mean: 114.9181 | STD: 653.1495 | Non-NaN values: 984
Qwen/Qwen2.5-14B-Instruct_int8_gps_fr_distance | Mean: 137.5599 | STD: 641.0181 | Non-NaN values: 975
Qwen/Qwen2.5-14B_int8_gps_fr_distance | Mean: 137.7392 | STD: 695.5877 | Non-NaN values: 966

💀 **Top 5 Worst Columns** (Highest Mean):
meta-llama/Llama-2-13b-chat-hf_float16_gps_fr_distance | Mean: 6127.1839 | STD: 4790.6469 | Non-NaN values: 971
meta-llama/Llama-3.1-8B-Instruct_int8_gps_fr_distance | Mean: 6050.3589 | STD: 5063.5476 | Non-NaN values: 757
meta-llama/Llama-2-13b-chat-hf_int8_gps_fr_distance | Mean: 5775.2845 | STD: 4697.7379 | Non-NaN values: 976
meta-llama/Llama-2-13b-chat-hf_int4_gps_fr_distance | Mean: 5566

In [ ]:
distance_columns = [col for col in prediction.columns if "distance" in col]
distance_means = [prediction[col].mean() for col in distance_columns]
df = pd.DataFrame({
    "distance_mean": distance_means,
    "model_family": [col.lower().split("/")[-1].rsplit("-",1)[0] for col in distance_columns],
    "model_size": [re.search(r'(\d+\.?\d*B)', col, re.VERBOSE | re.IGNORECASE).group(1) for col in distance_columns],
    "quantization": [col.split("_")[1] for col in distance_columns],
    "lang_prompt": [col.split("_")[3] for col in distance_columns],
    "prompt_type": [col.split("_")[2] for col in distance_columns],
    # "fine_tuning": [col.split("_")[5] for col in distance_columns]
})
df

,distance_mean,model_family,model_size,quantization,lang_prompt,prompt_type
0,146.241734,mistral-7,7B,float16,fr,gps
1,363.840434,mistral-7,7B,float16,en,gps
2,166.194266,mistral-7,7B,int8,fr,gps
3,378.279004,mistral-7,7B,int8,en,gps
4,199.067712,mistral-7,7B,int4,fr,gps
...,...,...,...,...,...,...
121,279.694718,qwen2.5-14,14B,int8,en,gps
122,832.009506,qwen2.5-14,14B,int4,fr,gps
123,332.384067,qwen2.5-14,14B,int4,en,gps
124,156.692299,qwen2.5-32,32B,float16,fr,gps


In [64]:
re.search(r'-(?P<params>\d+B)', col, re.VERBOSE | re.IGNORECASE).groups()

AttributeError: 'NoneType' object has no attribute 'groups'

In [50]:
import re
pattern = re.compile(r'^(?P<model>[^/]+)/(?P<name>[^-]+)-(?P<params>\d+B)(-v\d+)?_(?P<quant>float16|int[48])_(?P<prompt>gps_[a-z]+)_distance$')

# List to store the extracted components
extracted_data = []

# Iterate over the list of model strings and extract the components
for model in distance_columns:
    match = pattern.match(model)
    if match:
        model_name = match.group('model')
        model_base = match.group('name')
        params = match.group('params')
        quantization = match.group('quant')
        prompt = match.group('prompt')
        extracted_data.append((model_name, model_base, params, quantization, prompt))

# Print the extracted data
for data in extracted_data:
    print(data)

('Qwen', 'Qwen2.5', '7B', 'float16', 'gps_fr')
('Qwen', 'Qwen2.5', '7B', 'float16', 'gps_en')
('Qwen', 'Qwen2.5', '7B', 'int8', 'gps_fr')
('Qwen', 'Qwen2.5', '7B', 'int8', 'gps_en')
('Qwen', 'Qwen2.5', '7B', 'int4', 'gps_fr')
('Qwen', 'Qwen2.5', '7B', 'int4', 'gps_en')
('Qwen', 'Qwen2.5', '14B', 'float16', 'gps_fr')
('Qwen', 'Qwen2.5', '14B', 'float16', 'gps_en')
('Qwen', 'Qwen2.5', '14B', 'int8', 'gps_fr')
('Qwen', 'Qwen2.5', '14B', 'int8', 'gps_en')
('Qwen', 'Qwen2.5', '14B', 'int4', 'gps_fr')
('Qwen', 'Qwen2.5', '14B', 'int4', 'gps_en')
('Qwen', 'Qwen2.5', '32B', 'float16', 'gps_fr')
('Qwen', 'Qwen2.5', '32B', 'float16', 'gps_en')


In [36]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Sélectionner les colonnes contenant "distance"
distance_columns = [col for col in prediction.columns if "distance" in col]

# Calculer la moyenne des distances pour chaque colonne
distance_means = [prediction[col].mean() for col in distance_columns]

# Créer un DataFrame avec ces moyennes et les critères
df = pd.DataFrame({
    "distance_mean": distance_means,
    "model_family": [col.split("_")[0] for col in distance_columns],
    "model_size": [col.split("_")[1] for col in distance_columns],
    "quantization": [col.split("_")[2] for col in distance_columns],
    "lang_prompt": [col.split("_")[3] for col in distance_columns],
    "prompt_type": [col.split("_")[4] for col in distance_columns],
    # "fine_tuning": [col.split("_")[5] for col in distance_columns]
})

# ANOVA à plusieurs facteurs avec un modèle linéaire
# formula = "distance_mean ~ C(model_family) + C(model_size) + C(quantization) + C(lang_prompt) + C(prompt_type) + C(fine_tuning)"
formula = "distance_mean ~ C(model_family) + C(model_size) + C(quantization) + C(lang_prompt) + C(prompt_type)"
model = smf.ols(formula, data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)  # ANOVA Type II

# Afficher les résultats
print("📊 **ANOVA Results on Means of Distances:**")
print(anova_table)

# Identifier le critère le plus significatif (plus petite p-value)
most_significant = anova_table["PR(>F)"].idxmin()
print(f"\n✅ The most significant factor is: **{most_significant}** (p-value = {anova_table.loc[most_significant, 'PR(>F)']:.4e})")


ValueError: must have at least one row in constraint matrix